In [18]:
# Import Dependencies.

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [19]:
# Create a set of random latitude and longitude combinations.

latitudes = np.random.uniform(low = -90, high = 90, size = 1500)
longitudes = np.random.uniform (low = -180, high = 180, size = 1500)
latitudes_longitudes = zip(latitudes,longitudes)
latitudes_longitudes

In [20]:
# Unpack the longitudes and latitudes and put them in a list. 

coordinates = list(latitudes_longitudes)
len(coordinates)

1500

In [21]:
# Nearest cities for the above coordinates.

# Create a empty list that will hold all the cities name nearest to the above coordinates.
cities = []
# Identify the nearest city for each latitude and longitutde combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    # if city is unique then add to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)



629